## 머신러닝 실습

### 텍스트 마이닝

#### 데이터 수집

##### 네이버 코로나 뉴스로 감성분석

In [ ]:
# 필요 라이브러리 등록
import json

In [ ]:
with open('./data/코로나_naver_news.json', encoding = 'utf-8') as f:
    data = json.load(f)

In [ ]:
import pandas as pd
import re

In [ ]:
dfData = pd.DataFrame(data)

In [ ]:
dfData.head()

In [ ]:
## 한글 이외의 것 다 제거하는 작업
dfData['title'] = dfData['title'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+',' ', x))

In [ ]:
dfData['description'] = dfData['description'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+',' ', x))

In [ ]:
dfData[['title','pDate','description']].to_excel('./data/코로나뉴스_전처리.xlsx',index = False)

In [ ]:
dfData = pd.read_excel('./data/코로나뉴스_전처리.xlsx')

In [ ]:
dfData.head()

##### 감성분석 모델 재구축

In [ ]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('../day12/data/ratings_train.txt',engine='python', sep='\t', encoding='utf-8')

In [ ]:
dfNsmcTrain.head()

In [ ]:
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'].notnull()]

In [ ]:
## 한글이외에 다 제거
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^가-힣|ㄱ-]+', ' ' , x))

In [ ]:
## 한글 제거 후에 다시 document가 빈 row를 제거
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'] != ' ']

In [ ]:
## 벡터화 , 로지스틱회귀 모듈 등록
import konlpy
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
okt = Okt()

In [ ]:
def oktToken(text):
    tokens = okt.morphs(text)
    return tokens

In [ ]:
## 벡터화 객체 생성
tfidf = TfidfVectorizer(tokenizer=oktToken, ngram_range=(1,2), min_df=3,max_df=0.9)

In [ ]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

In [ ]:
## 로지스틱회귀 모델 생성
model = LogisticRegression(random_state=0, C=3.5)

In [ ]:
model.get_params()

In [ ]:
model.fit(nsmc_train_tfidf,dfNsmcTrain['label'])

##### 네이버 뉴스 타이틀로 감성 분석

In [ ]:
# 분석할 데이터의 백터화
data_title_tfidf = tfidf.transform(dfData['title'])

In [ ]:
# 감성분석(로지스틱회귀)
title_predict = model.predict(data_title_tfidf)

In [ ]:
# 감성분석 결과 DF 저장
dfData['title_label'] = title_predict

In [ ]:
# 설명도 동일하게 감성분석
data_desc_tfidf = tfidf.transform(dfData['description'])

In [ ]:
# 감성분석
dfData['description_label'] = model.predict(data_desc_tfidf)

In [ ]:
dfData.tail()

In [ ]:
dfData.to_excel('./data/코로나뉴스_감성분석_결과.xlsx',index=False)

##### 시각화 준비

In [ ]:
dfResult = pd.read_excel('./data/코로나뉴스_감성분석_결과.xlsx')

In [ ]:
dfResult.info()

In [ ]:
# 타이틀의 감성별 갯수
dfResult['title_label'].value_counts()

In [ ]:
# 기사 내용의 감성별 갯수
dfResult['description_label'].value_counts()

In [ ]:
# 긍정적 결과와 부정적 결과를 분리
column_names = dfResult.columns.tolist()

In [ ]:
column_names.remove('pDate')

In [ ]:
column_names

In [ ]:
# 빈 DF생성
dfPositiveData = pd.DataFrame(columns=column_names)
dfNegativeData = pd.DataFrame(columns=column_names)

# description_label이 긍정인지 부정인지 따라 dfPositiveDAta와 dfNegativeData에 나눠서 할당
for i, data in dfData.iterrows():
    title = data['title']
    descripition = data['description']
    title_label = data['title_label']
    description_label = data['description_label']

    if description_label == 1: # 긍정감성
        dfTemp = pd.DataFrame([[title,descripition,description_label,title_label]],columns=dfPositiveData.columns)
        dfTemp.columns = column_names
        dfPositiveData = pd.concat([dfPositiveData, dfTemp])
    else:   # 부정감성데이터
        dfTemp = pd.DataFrame([[title,descripition,description_label,title_label]],columns=dfPositiveData.columns)
        dfNegativeData = pd.concat([dfNegativeData,dfTemp])

In [ ]:
len(dfPositiveData),len(dfNegativeData)

##### 시각화


In [ ]:
# 긍정 데이터에서 단어 추출
posDescriptions = dfPositiveData['description']

In [ ]:
posDescriptionsWords = []

for d in posDescriptions:
    posDescriptionsWords.append(okt.nouns(d))   # 명사 형태소만 추출


In [ ]:
finalPosDescWords = []

for d in posDescriptionsWords:
    d2 = [w for w in d if len(w) > 1]   # 글자길이가 1보다 큰것만 추출
    finalPosDescWords.append(' '.join(d2))

In [ ]:
finalPosDescWords

In [ ]:
## TF-IDF기반 단어별 출현빈도 계산
posTfidf =  TfidfVectorizer(tokenizer=oktToken,min_df=2)

In [ ]:
posDtm =  posTfidf.fit_transform(finalPosDescWords)

In [ ]:
posVocas = dict()
for i, word in enumerate(posTfidf.get_feature_names_out()):
   posVocas[word] = posDtm.getcol(i).sum() # 해당 단어의 빈도수

In [ ]:
# 빈도수가 높은 것 부터 정렬
posWord = sorted(posVocas.items(), key=(lambda x : x[1]), reverse=True)

In [ ]:
# 차트 라이브러리 등록
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style('darkgrid')

In [ ]:
# 한글 깨짐 문제 해결
from matplotlib import rcParams, font_manager, rc

font_path = 'C:/Windows/Fonts/malgun.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font',family=font)
rcParams['axes.unicode_minus'] = False

In [ ]:

max = 15
plt.figure(figsize = (15,5))
plt.bar(range(max),[i[1] for i in posWord[:max]], color='blue')
plt.title('네이버 긍정뉴스 단어 상위 15개', fontsize=14)
plt.xlabel('단어', fontsize=12)
plt.ylabel('빈도수',fontsize=12)
# x축 단어 표시
plt.xticks(range(max),[i[0] for i in posWord[:max]])

plt.show()
